In [151]:
class Token:
    def __init__(self, value: str, type: TokenType):
        self.value = value
        self.type = type

In [153]:
from enum import Enum

class TokenType(Enum):
    NUMBER = "number"
    OPERATOR = "operator" 
    PARENTHESIS = "parenthesis"

In [155]:
def get_token_type(token: str) -> TokenType:
    try:
        value = int(token)
        return TokenType.NUMBER
    except ValueError:
        pass

    token_definitions = {
        ('.', ',', 'e'):  TokenType.NUMBER,
        ('(', ')'): TokenType.PARENTHESIS,
        ('^', '*', '/', '+', '-'): TokenType.OPERATOR
    }  
    
    for chars, token_type in token_definitions.items():
        if token in chars:
            return token_type

    return None

In [157]:
def tokenize(uin: str) -> list[Token]:
    tokens = []
    prev_num = ''
    
    for c in uin:
        type = get_token_type(c)
        
        if type == None:
            raise ValueError(f'Invalid character: {c}')
        if type == TokenType.NUMBER or prev_num != '' and prev_num[-1] == 'e':
            prev_num += c
        else:
            if prev_num != '':
                num = convert_num(prev_num)
                tokens.append(Token(value=num, type=TokenType.NUMBER))
                prev_num = ''
            tokens.append(Token(value=c, type=type))
            
    if prev_num != '':
        num = convert_num(prev_num)
        tokens.append(Token(value=num, type=TokenType.NUMBER))
        
    return tokens

In [159]:
def validate_parenthesis(tokens: list[Token]) -> bool:
    stack = []
    for token in tokens:
        if token.value == '(':
            stack.append(token.value)
        elif token.value == ')':
            if not stack or stack.pop() != '(':
                raise ValueError
    if stack:
        raise ValueError

In [161]:
def convert_num(num: str) -> float:
    if 'e' in num or 'E' in num:
        expr= f"{float(num):.1e}".replace('+', '')
        expr = expr.replace('E', 'e')
        
        base, exponent = expr.lower().split('e')
        base = float(base)
        exponent = int(exponent)
        result = base * (10 ** exponent)
        
        return result
        
    num = float(num)
    if num.is_integer():
        num = int(num)
    return num

In [163]:
def infix_to_postfix(tokens: list[Token]) -> list[Token]:
    la = {'+': 1, '-': 1, '*': 2, '/': 2, '^': 3} # left-associative
    ra = {'^'} # right-associative
    output = [] # Token
    operators = [] # Token

    for token in tokens:
        if token.type == TokenType.NUMBER:
            output.append(token)
        elif token.value in la:
            while (operators and operators[-1].value in la and
                  (la[operators[-1].value] > la[token.value] or
                  (la[operators[-1].value] == la[token.value] and token.value not in ra))):
                output.append(operators.pop())
            operators.append(token)
        elif token.value == '(':
            operators.append(token)
        elif token.value == ')':
            while operators and operators[-1].value != '(':
                output.append(operators.pop())
            operators.pop()
        else:
            raise ValueError(f"Unknown token: {token.value}")

    while operators:
        output.append(operators.pop())

    return output

In [169]:
operations = {
    '+': lambda x, y: x + y,
    '-': lambda x, y: x - y,
    '*': lambda x, y: x * y,
    '/': lambda x, y: x / y if y != 0 else 'NaN',
    '^': lambda x, y: x ** y
}

In [171]:
def evaluate_postfix(postfix_tokens: list[Token]) -> float:
    stack = []
    for token in postfix_tokens:
        if token.type == TokenType.NUMBER:
            stack.append(token.value)
        else:
            operand2 = stack.pop()
            operand1 = stack.pop()
            result = operations[token.value](operand1, operand2)
            stack.append(result)

    return stack[0]

In [173]:
from typing import Any

def calculate_result(tokens: list[Any]) -> float:
    if len(tokens) == 0:
        return 0
    postfix_tokens = infix_to_postfix(tokens)
    result = evaluate_postfix(postfix_tokens)
    if isinstance(result, float) and result.is_integer():
        result = int(result)
    return result

In [175]:
def calculator(history: list[dict]):
    uin = input('\nENTER OPERATION: ').strip()
    if uin in history:
        result = history[uin]
    else:
        tokens = tokenize(uin)
        validate_parenthesis(tokens)
        result = calculate_result(tokens)
        print(result)
        history[uin] = result
    return history

In [197]:
import sympy as sp

def sympy_verifier(expr: str) -> float:
    try:
        expr = expr.lstrip('0') or '0'
        expr = expr.replace('^', '**')
        result = sp.sympify(expr).evalf(16)
        try:
            result = float(result)
            if result.is_integer():
                result = int(result)
        except ValueError:
            pass
        return result
    except (sp.SympifyError, TypeError):
        raise ValueError('NaN')

In [199]:
test_cases = [
    "",
    ".",
    "1a",
    ".0",
    "1/0",
    "2e2",
    "5e-10",
    "000004",
    "4^(2/4)",
    "0.0000000000001+1",
    "0.00000000000000005*0",
    "(50.5 + 49.5) * 2 - 100",
    "1000 - 500.0001 * 2 + 300 / 3",
    "8.5 * 3.2 + 2.1 ^ 4 - (6.3 / 3.1)",
    "5.5 / 1.1 + 4.4 * (3.3 ^ 2) - 6.6",
    "2.2 ^ 4 + 3.3 * 5.5 - (8.8 / 2.2)",
    "7.7 + 8.8 * (5.5 - 3.3) ^ 2 / 4.4",
    "(9.9 + 3.3) * 2.2 - 4.4 ^ 2 + 7.7",
    "6.6 / (2.2 + 3.3 * (8.8 - 4.4) ^ 2)",
    "3.3 * 4.4 + 2.2 ^ (5/2) - (7.7 - 1.1)",
    ".200 / 2 + 3.5 * (4.2 + 5.8) - 60 ^ 2",
    "123.45000 + 678.9 * 2.1 - 345.67 / 1.2",
    "1.79e308+1",
    "9999999999999999999999999+9999999999999999999",
]

In [201]:
import pandas as pd
def display_history(history: list[dict]):
    df = pd.DataFrame(history)
    pd.set_option('display.float_format', lambda x: f'{x:.10f}')
    display(df)

In [203]:
history = []
test_cases = [case.replace(" ", "") for case in test_cases]

for t in test_cases:
    if t not in history:
        try:
            tokens = tokenize(t)
            validate_parenthesis(tokens)
            result = calculate_result(tokens)
        except ValueError:
            result = 'NaN'
        try:
            expected = sympy_verifier(t)
        except ValueError:
            expected = 'NaN'
        history.append({'Input': t, 'Result': result, 'Expected': expected})

display_history(history)

,Input,Result,Expected
0,,0,0
1,.,NaN,NaN
2,1a,NaN,NaN
3,.0,0,0
4,1/0,NaN,NaN
5,2e2,200,200
6,5e-10,0.0000000005,0.0000000005
7,000004,4,4
8,4^(2/4),2,2
9,0.0000000000001+1,1.0000000000,1.0000000000


In [205]:
from IPython.display import clear_output
import pandas as pd

def main():
    uin = '-1'
    history = {}
    while uin != '0':
        print('WELCOME')
        print('-'*20)
        print('0 - EXIT\n1 - CALCULATOR\n2 - HISTORY')
        print('-'*20)
        uin = input('\nSELECT AN OPTION: ')
        if uin == '1':
            history = calculator(history)
        elif uin == '2':
            display_history(history)
        elif uin == '0':
            return None
        else:
            raise ValueError
        input()
        clear_output()

In [207]:
main()

WELCOME
--------------------
0 - EXIT
1 - CALCULATOR
2 - HISTORY
--------------------



SELECT AN OPTION:  0
